In [41]:
import json
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
with open('data/case-study-ds-sa.json') as f:
    info = json.load(f)
client = bigquery.Client.from_service_account_info(info)

In [16]:
for table in ["test2", "test3", "test4"]:
    q = f"""
    SELECT 
        COUNT(DISTINCT consolidated_user_id) AS num_users,
        COUNT(*) AS num_events
    FROM `tmp-case-study-ds.case_study.{table}`
    """
    res = client.query(q).to_dataframe(create_bqstorage_client=False) #API Storage refusé, je dois prendre REST
    
    print(f"Table {table}: {res['num_users'][0]} utilisateurs, {res['num_events'][0]} événements")

Table test2: 42699 utilisateurs, 28779003 événements
Table test3: 16750 utilisateurs, 6596362 événements
Table test4: 16870 utilisateurs, 48181829 événements


In [18]:
# Regardons juste une ligne pour voir les noms de colonnes
check_cols = "SELECT * FROM `tmp-case-study-ds.case_study.test4` LIMIT 1"
df_check = client.query(check_cols).to_dataframe(create_bqstorage_client=False)
print(df_check.columns)

Index(['consolidated_user_id', 'domain_id', 'visit_id', 'visit_date',
       'local_start_time', 'local_end_time', 'time_spent_seconds',
       'product_name', 'main_feature'],
      dtype='object')


In [ ]:
sql_get_domains = """
SELECT DISTINCT domain_id 
FROM `tmp-case-study-ds.case_study.test4` 
WHERE domain_id IS NOT NULL
"""
domains_df = client.query(sql_get_domains).to_dataframe(create_bqstorage_client=False)
domain_list = domains_df['domain_id'].tolist()

print(f"Métiers trouvés : {domain_list}")

Métiers trouvés : ['INFORMATIONSYSTEM', 'HUMANRESOURCES', 'GUIDANCE', 'OFFRE', 'RETAIL', 'WEB', 'DIGITALMERCH', 'COMMUNICATION', 'FINANCE', 'REALESTATE', 'LEGALCOMP', 'PASSION', 'INDUSTRY', 'LOGISTIC']


In [35]:
for domain in domain_list : 

    q_metier = """
    SELECT 
        domain_id, 
        product_name, 
        COUNT(*) as nombre_utilisations
    FROM `tmp-case-study-ds.case_study.test4`
    GROUP BY domain_id, product_name
    ORDER BY domain_id, nombre_utilisations DESC
    """

    df_metier = client.query(q_metier).to_dataframe(create_bqstorage_client=False)
    df_metier.head()
    # Pour voir le résultat pour un métier spécifique (ex: MARKETING)
    print(df_metier[df_metier['domain_id'] == f'{domain}'].head(5))
    print('-'*50)

              domain_id     product_name  nombre_utilisations
3157  INFORMATIONSYSTEM            Gmail              2727198
3158  INFORMATIONSYSTEM  Google Calendar              1297745
3159  INFORMATIONSYSTEM      Google Meet               946591
3160  INFORMATIONSYSTEM    Google Sheets               943187
3161  INFORMATIONSYSTEM      Google Chat               715215
--------------------------------------------------
           domain_id     product_name  nombre_utilisations
1807  HUMANRESOURCES            Gmail               633387
1808  HUMANRESOURCES    Google Sheets               359629
1809  HUMANRESOURCES  Google Calendar               247615
1810  HUMANRESOURCES    Google Slides               193640
1811  HUMANRESOURCES     Google Drive               143538
--------------------------------------------------
     domain_id     product_name  nombre_utilisations
1509  GUIDANCE            Gmail               177761
1510  GUIDANCE    Google Sheets                83831
1511  GUIDANC

In [43]:
def save_workflow_chart(df, domaine):
    plt.figure(figsize=(10, 6))
    # On prend le top 10
    top_df = df.head(10).sort_values('frequency', ascending=True) # Tri inversé pour l'affichage
    
    # Création du barplot horizontal
    bars = plt.barh(top_df['workflow_chain'], top_df['frequency'], color='#4c72b0')
    
    plt.title(f"Top 10 Workflows - {domaine}", fontsize=14)
    plt.xlabel("Fréquence (Nombre de sessions)", fontsize=12)
    plt.tight_layout()
    
    # Sauvegarde
    filename = f"chart_{domaine}.png"
    plt.savefig(filename)
    plt.close() # On ferme pour ne pas surcharger la mémoire
    print(f"Graphique sauvegardé : {filename}")

In [44]:
# Dictionnaire pour stocker les résultats si tu veux faire des graphiques après
all_results = {}

# 2. La Boucle Automatique
for domaine in domain_list:
    print(f"\n ANALYSE DU DOMAINE : {domaine}")
    
    # La fameuse requête "Full Stack" (Nettoyage + Session + Workflow)
    query_loop = f"""
    WITH raw_data AS (
        SELECT 
            consolidated_user_id,
            product_name,
            local_start_time,
            local_end_time,
            -- On regarde l'app d'avant pour dédoublonner
            LAG(product_name) OVER (PARTITION BY consolidated_user_id ORDER BY local_start_time) as prev_app,
            LAG(local_end_time) OVER (PARTITION BY consolidated_user_id ORDER BY local_start_time) as prev_end
        FROM `tmp-case-study-ds.case_study.test4`
        WHERE domain_id = '{domaine}'
    ),
    filtered_events AS (
        -- On supprime les répétitions immédiates (A -> A) et on calcule le gap
        SELECT *,
            TIMESTAMP_DIFF(local_start_time, prev_end, SECOND) as gap
        FROM raw_data
        WHERE product_name != prev_app OR prev_app IS NULL
    ),
    sessionized AS (
        -- On coupe si > 30 min
        SELECT *,
            SUM(CASE WHEN gap > 1800 OR gap IS NULL THEN 1 ELSE 0 END) 
            OVER (PARTITION BY consolidated_user_id ORDER BY local_start_time) as session_id
        FROM filtered_events
    ),
    workflows AS (
        -- On crée la chaîne
        SELECT
            consolidated_user_id,
            session_id,
            STRING_AGG(product_name, ' -> ' ORDER BY local_start_time) as workflow_chain
        FROM sessionized
        GROUP BY consolidated_user_id, session_id
    )
    SELECT 
        workflow_chain,
        COUNT(*) as frequency
    FROM workflows
    GROUP BY workflow_chain
    ORDER BY frequency DESC
    LIMIT 20
    """
    
    # Exécution
    try:
        df_res = client.query(query_loop).to_dataframe(create_bqstorage_client=False)
        all_results[domaine] = df_res
        print(df_res)

        if not df_res.empty:
            save_workflow_chart(df_res, domaine)
            
    except Exception as e:
        print(f"Erreur sur le domaine {domaine}: {e}")

print("\n Analyse terminée pour tous les métiers.")


 ANALYSE DU DOMAINE : INFORMATIONSYSTEM
                       workflow_chain  frequency
0                               Gmail       2446
1                     Google Calendar       1369
2                         Google Chat        854
3                               COM2U        744
4                              Github        730
5                       Google Gemini        647
6            Gmail -> Google Calendar        482
7                      COM2U -> Gmail        481
8                       Google Sheets        455
9            Google Calendar -> Gmail        446
10                               Jira        270
11                        Google Meet        255
12               Google Chat -> Gmail        229
13  Gmail -> Google Calendar -> Gmail        224
14                      Google Slides        180
15                            ChatGPT        179
16                        Google Docs        176
17     Google Calendar -> Google Meet        158
18                         C